In [1]:
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
test_size = 0.3
x=np.load('loaded_data/meld_full_x.npy') 
y=np.load('loaded_data/meld_full_y.npy') 
# print(set(y))
d, j = {}, 0
for i in set(y):
    d[i] = j
    j+=1

x_train,x_test,temp_y_train,temp_y_test=train_test_split(np.array(x), y, test_size = test_size, random_state = 9)


In [3]:
x_traincnn = np.expand_dims(x_train, axis=2)
x_testcnn = np.expand_dims(x_test, axis=2)

In [4]:
x_traincnn.shape, x_testcnn.shape

((6991, 180, 1), (2997, 180, 1))

In [5]:
y_test,y_train = [],[]
print(d)
for i in temp_y_test:
    y_test.append(d[i])
for i in temp_y_train:
    y_train.append(d[i])



{'surprise': 0, 'disgust': 1, 'fear': 2, 'sad': 3, 'neutral': 4, 'happy': 5, 'anger': 6}


In [6]:
y_train=np.array(y_train)
y_test=np.array(y_test)

In [7]:
import keras
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from tensorflow.keras.utils import to_categorical
from keras.layers import Input, Flatten, Dropout, Activation
from keras.layers import Conv1D, MaxPooling1D
from keras.models import Model
from keras.callbacks import ModelCheckpoint

In [10]:
model = Sequential()

model.add(Conv1D(128, 5,padding='same',
                 input_shape=(180,1)))
model.add(Activation('relu'))
model.add(Dropout(0.1))
model.add(MaxPooling1D(pool_size=(10)))
model.add(Conv1D(128, 5,padding='same',))
model.add(Activation('relu'))
model.add(Dropout(0.1))
model.add(Flatten())
model.add(Dense(7))
model.add(Activation('softmax'))
opt = tf.keras.optimizers.RMSprop(learning_rate=0.00005, rho=0.9, epsilon=None, decay=0.0)


In [11]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_4 (Conv1D)           (None, 180, 128)          768       
                                                                 
 activation_6 (Activation)   (None, 180, 128)          0         
                                                                 
 dropout_4 (Dropout)         (None, 180, 128)          0         
                                                                 
 max_pooling1d_2 (MaxPooling  (None, 18, 128)          0         
 1D)                                                             
                                                                 
 conv1d_5 (Conv1D)           (None, 18, 128)           82048     
                                                                 
 activation_7 (Activation)   (None, 18, 128)           0         
                                                      

In [ ]:


model.compile(loss='sparse_categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])



In [ ]:
history=model.fit(x_traincnn, y_train, batch_size=16, epochs=1000, validation_data=(x_testcnn, y_test))

In [ ]:
# model.save("nn_jl_1.h5")


In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
    import tensorflow as tf 

    if tf.test.gpu_device_name(): 

        print('Default GPU Device:{}'.format(tf.test.gpu_device_name()))

    else:

       print("Please install GPU version of TF")